<a href="https://colab.research.google.com/github/AneeshPK7/Responsible_A.I/blob/main/Resp_AI_Lab_4(i).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Lab Assignment 4
## CNN for Image Classification

In [ ]:

# Q1 Imports
import os, numpy as np
from PIL import Image
from tqdm import tqdm
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import kagglehub


In [ ]:
import kagglehub

# Download latest version from Kaggle
path = kagglehub.dataset_download("ryanbadai/clothes-dataset")
print("Path to dataset files:", path)


DATA_DIR = path
IMAGE_SIZE = (64, 64)

images = []
labels = []

base_classes_dir = os.path.join(DATA_DIR, "Clothes_Dataset")
classes = sorted(os.listdir(base_classes_dir))

for cls in classes:
    cls_path = os.path.join(base_classes_dir, cls)

    if os.path.isdir(cls_path):
        for img_name in tqdm(os.listdir(cls_path)):
            img_path = os.path.join(cls_path, img_name)

            if os.path.isfile(img_path):
                img = Image.open(img_path).convert("RGB").resize(IMAGE_SIZE)
                images.append(np.array(img))
                labels.append(cls)

images = np.array(images)
labels = np.array(labels)

print("Images shape:", images.shape)
print("Labels shape:", labels.shape)




100%|██████████| 1.37G/1.37G [00:35<00:00, 42.0MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ryanbadai/clothes-dataset/versions/1


100%|██████████| 500/500 [00:07<00:00, 68.42it/s]

Images shape: (7500, 64, 64, 3)
Labels shape: (7500,)


In [ ]:

class ClothesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X/255.0, dtype=torch.float32)
        self.map = {c:i for i,c in enumerate(sorted(set(y)))}
        self.y = torch.tensor([self.map[i] for i in y])

    def __len__(self): return len(self.y)
    def __getitem__(self, i): return self.X[i].permute(2,0,1), self.y[i]


In [ ]:

class CNN(nn.Module):
    def __init__(self, convs, num_classes):
        super().__init__()
        layers=[]
        in_c=3
        for c in convs:
            layers += [nn.Conv2d(in_c,c,3,padding=1), nn.ReLU(), nn.MaxPool2d(2)]
            in_c=c
        self.conv=nn.Sequential(*layers)
        self.fc=nn.Sequential(nn.Flatten(), nn.Linear(in_c*8*8,128), nn.ReLU(), nn.Linear(128,num_classes))

    def forward(self,x): return self.fc(self.conv(x))


In [ ]:

def train_eval(model, train_loader, test_loader, epochs=8):
    opt=optim.Adam(model.parameters())
    loss_fn=nn.CrossEntropyLoss()
    for _ in range(epochs):
        for X,y in train_loader:
            opt.zero_grad()
            loss_fn(model(X),y).backward()
            opt.step()
    model.eval()
    p,t=[],[]
    with torch.no_grad():
        for X,y in test_loader:
            p+=model(X).argmax(1).tolist()
            t+=y.tolist()
    return f1_score(t,p,average='macro')


In [ ]:
Xtr,Xte,ytr,yte=train_test_split(images,labels,test_size=0.2,stratify=labels)
train_ds, test_ds = ClothesDataset(Xtr,ytr), ClothesDataset(Xte,yte)
train_dl, test_dl = DataLoader(train_ds,64,True), DataLoader(test_ds,64)

class CNN(nn.Module):
    def __init__(self, convs, num_classes, image_size=(64, 64)):
        super().__init__()
        layers=[]
        in_c=3
        for c in convs:
            layers += [nn.Conv2d(in_c,c,3,padding=1), nn.ReLU(), nn.MaxPool2d(2)]
            in_c=c # in_c now holds the number of output channels of the last conv layer
        self.conv=nn.Sequential(*layers)

        num_max_pools = len(convs)
        final_h = image_size[0] // (2 ** num_max_pools)
        final_w = image_size[1] // (2 ** num_max_pools)
        flattened_size = in_c * final_h * final_w

        self.fc=nn.Sequential(nn.Flatten(), nn.Linear(flattened_size,128), nn.ReLU(), nn.Linear(128,num_classes))

    def forward(self,x): return self.fc(self.conv(x))

configs={"1conv":[16],"2conv":[16,32],"3conv":[16,32,64]}
for k,v in configs.items():
    train_eval(CNN(v,len(set(labels)), IMAGE_SIZE),train_dl,test_dl)
    f1=train_eval(CNN(v,len(set(labels)), IMAGE_SIZE),train_dl,test_dl)
    print(k,f1)


1conv 0.46459169222983276
2conv 0.5138121440076799
3conv 0.517104475731715
